In [ ]:
import os
import pandas as pd
import numpy as np
import random
import ntpath
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

## Cargar los datos generados por el simulador de Udacity

In [ ]:
directorio = "C:\\Users\\XPC\\Desktop\\sdc\\"
columnas = ["centro","izquierda","derecha","manivela","potencia","reversa","velocidad"]
data = pd.read_csv(directorio + "\\driving_log.csv", names = columnas)
data.head()

## Limipiar las rutas del Dataframe

In [ ]:
# limpiar las rutas en el dataframe
def path_tail(path):
    _, tail = ntpath.split(path)
    return tail

data["centro"] = data["centro"].apply(path_tail)
data["izquierda"] = data["izquierda"].apply(path_tail)
data["derecha"] = data["derecha"].apply(path_tail)
data.head()

## El problema del Imbalance

In [ ]:
# el problema del imbalance en manivela.
muestras_por_bin = 200
numero_bins = 30
hist, bins = np.histogram(data["manivela"], numero_bins)
centrar = (bins[:-1] + bins[1:]) * 0.5
plt.bar(centrar, hist, width=0.05)
plt.axhline(y=300, color='r', linestyle='-')

# existen demasiados datos que apuntan en el centro (osea el carro siempre va recto)

In [ ]:
# la estrategia es eliminar una cantidad de datos tal que permita 
# tener una cantidad balanceada de cada elemento de la distribucion

# la linea roja sugiere conservar 200 de los datos del bin mas grande de forma aleatoria.

print("largo dataset:", len(data))
remover = []   
for j in range(numero_bins):
    lista = []
    for i in range(len(data["manivela"])):
        if(data["manivela"][i] >= bins[j] and data["manivela"][i] <= bins[j+1]):
            lista.append(i) #indice
    lista = shuffle(lista)
    lista = lista[muestras_por_bin:]
    remover.extend(lista)
        
print("Remover:", len(remover))
data.drop(data.index[remover], inplace=True)
print("nuevo largo dataset:", len(data))

In [ ]:
# revisamos la nueva distribucion despues de remover los datos innecesarios.

hist, _ = np.histogram(data["manivela"], numero_bins)
plt.bar(centrar, hist, width=0.05)

In [ ]:
# salvamos el nuevo dataset de entrenamiento
data.to_csv(directorio + "\\driving_log_limpio.csv", index=False)